In [27]:
import numpy as np

x = np.array(([3,5],[5,1],[10,2]),dtype=float)
y = np.array(([75],[82],[93]),dtype=float)
x = x/np.amax(x,axis=0)
y = y/100

class Neural_Network(object):
    def __init__(self):
        self.inputLayerSize = 2
        self.outputLayerSize = 1
        self.hiddenLayerSize = 3
        self.W1 = np.random.randn(self.inputLayerSize,self.hiddenLayerSize)
        self.W2 = np.random.randn(self.hiddenLayerSize,self.outputLayerSize)
        
    def forward(self,x):
        self.z2 = np.dot(x,self.W1)
        self.a2 = self.sigmoid(self.z2)
        self.z3 = np.dot(self.a2,self.W2)
        yHat = self.sigmoid(self.z3)
        return yHat
    
    def sigmoid(self, z):
        return 1/(1+np.exp(-z))
    
    def sigmoidPrime(self,z):
        return np.exp(-z)/((z+np.exp(-z)**2))
    
    def costFunction(self, x, y):
        self.yHat = self.forward(x)
        J = 0.5*sum((y-self.yHat)**2)
        return J
    
    def costFunctionPrime(self, x,y):
        self.yHat = self.forward(x)
        delta3 = np.multiply(-(y-self.yHat),self.sigmoidPrime(self.z3))
        dJdW2 = np.dot(self.a2.T,delta3)
        delta2 = np.dot(delta3,self.W2.T)*self.sigmoidPrime(self.z2)
        dJdW1 = np.dot(x.T,delta2)
        return dJdW1, dJdW2
    
    def getParams(self):
        params = np.concatenate((self.W1.ravel(), self.W2.ravel()))
        return params
    
    def setParams(self, params):
        W1_start = 0
        W1_end = self.hiddenLayerSize * self.inputLayerSize
        self.W1 = np.reshape(params[W1_start:W1_end], (self.inputLayerSize , self.hiddenLayerSize))
        W2_end = W1_end + self.hiddenLayerSize*self.outputLayerSize
        self.W2 = np.reshape(params[W1_end:W2_end], (self.hiddenLayerSize, self.outputLayerSize))
        
    def computeGradients(self, x, y):
        dJdW1, dJdW2 = self.costFunctionPrime(x, y)
        return np.concatenate((dJdW1.ravel(), dJdW2.ravel()))

In [28]:
def computeNumericalGradient(N, x, y):
        paramsInitial = N.getParams()
        numgrad = np.zeros(paramsInitial.shape)
        perturb = np.zeros(paramsInitial.shape)
        e = 1e-4
        for p in range(len(paramsInitial)):
            perturb[p] = e
            N.setParams(paramsInitial + perturb)
            loss2 = N.costFunction(x, y)
            N.setParams(paramsInitial - perturb)
            loss1 = N.costFunction(x, y)
            numgrad[p] = (loss2 - loss1) / (2*e)
            perturb[p] = 0
        N.setParams(paramsInitial)

        return numgrad

In [29]:
NN = Neural_Network()

In [30]:
numgrad = computeNumericalGradient(NN, x, y)
numgrad

array([ 0.00289235,  0.00338689, -0.02968758,  0.00217045,  0.0025445 ,
       -0.02010283, -0.05331151, -0.04453445, -0.08029606])

In [31]:
grad = NN.computeGradients(x,y)
grad

array([ 0.01891282,  0.02286135, -0.35894266,  0.01627352,  0.02082263,
       -0.24513126, -0.16566778, -0.13773311, -0.24670071])

In [32]:
norm(grad-numgrad)/norm(grad+numgrad)

NameError: name 'norm' is not defined